In [83]:
#Read in libraries
import pandas as pd
import swifter

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn

In [84]:
#Set style for matplotlib
plt.style.use('ggplot')

In [85]:
#Set options for pandas
pd.options.display.max_columns =1_000
pd.set_option('display.max_rows', 1_000)

#Surpress future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [86]:
#Read in data

path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\04_Models'

#Read in listings data
listings = pd.read_csv(path + '/01_19_2020_Listings_Processed.csv',  index_col=0)

#Ordinal variables to convert to category
ordinals = ['review_scores_accuracy', 'review_scores_checkin','review_scores_cleanliness', 
            'review_scores_communication','review_scores_location','review_scores_rating',
            'review_scores_value']

#Convert 
listings[ordinals] = listings[ordinals].astype('category')

# Preview Data

In [87]:
print('Listings shape: ', listings.shape)
listings.head()

Listings shape:  (65767, 469)


,accommodates,availability_30,availability_365,bathrooms,bedrooms,beds,chlc,chlc_private_rooms,chlc_shared_rooms,cleaning_fee,extra_people,guests_included,host_identity_verified,host_is_superhost,host_response_rate,instant_bookable,is_location_exact,maximum_maximum_nights,number_of_reviews,number_of_reviews_ltm,price,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,security_deposit,total_amenities,24-hour,Accessible-height,Air,Amazon,BBQ,Baby,Babysitter,Balcony,Bath,Bathroom,Bathtub,Beach,Beachfront,Bed,Bedroom,Bidet,Body,Breakfast,Building,Buzzer/wireless,Cable,Carbon,Cat(s),Ceiling,Central,Changing,Children’s,Cleaning,Coffee,Convection,Cooking,Crib,DVD,Day,Disabled,Dishes,Dishwasher,Dog(s),Doorman,Double,Dryer,Echo,Electric,Elevator,Espresso,Essentials,Ethernet,Exercise,Extra,Family/kid,Fax,Fire,Fireplace,Firm,First,Fixed,Flat,Formal,Free,Full,Game,Garden,Gas,Ground,Gym,HBO,Hair,Hammock,Handheld,Hangers,Heat,Heated,Heating,High,High-resolution,Host,Hot,Indoor,Internet,Iron,Jetted,Keypad,Kitchen,Kitchenette,Lake,Laptop,Lock,Lockbox,Long,Luggage,Memory,Microwave,Mini,Mobile,Mountain,Mudroom,Murphy,Netflix,Other,Outdoor,Outlet,Oven,Pack,Paid,Patio,Pets,Pillow-top,Play/travel,Pocket,Pool,Printer,Private,Projector,Rain,Refrigerator,Roll-in,Room-darkening,Safety,Self,Shampoo,Shared,Shower,Single,Ski-in/Ski-out,Smart,Smoke,Smoking,Soaking,Sound,Stair,Stand,Standing,Step-free,Stove,Suitable,Sun,Table,Tennis,Terrace,Toilet,Walk-in,Warming,Washer,Waterfront,Well-lit,Wheelchair,Wide,Wifi,Window,Wine,access,accessible,aid,air,allowed,alone,and,area,around,backyard,balcony,bars,basics,bath,bathroom,bed,bedroom,before,blankets,books,card,chair,charger,check-in,checkout,clearance,comforts,computer,conditioning,connection,console,cooler,corner,court,covers,crib,detector,dining,dinnerware,door,doorway,drawer,dropoff,dryer,en.hosting_amenity_49,en.hosting_amenity_50,enter,entrance,entryway,equipment,essentials,events,extinguisher,fan,fireplace,floor,floors,foam,for,fridge,friendly,front,gates,grab,greets,grill,guards,guest,guests,gym,hallway,hallways,head,hoist,home,hot,intercom,kettle,kit,kitchen,lamps,level,linens,live,living,lock,loungers,machine,maker,mattress,missing:,monitor,monoxide,off,oven,paper,parking,path,pet(s),pillows,pit,player,pool,premises,profiling,property,rack,recommendations,room,screen,seating,shades,shower,silverware,soap,space,spot,staff,stairs,stays,steam,steps,street,suite,system,table,term,this,toilet,towel,toys,translation,tub,valet,view,water,wifi,with,workspace,you,None,email,facebook,google,government_id,identity_manual,jumio,kba,manual_offline,manual_online,offline_government_id,phone,photographer,reviews,selfie,sent_id,weibo,work_email,zhima_selfie,host_identity_verified.1,host_is_superhost.1,instant_bookable.1,is_location_exact.1,require_guest_phone_verification.1,require_guest_profile_picture.1,requires_license.1,bed_type_Airbed,bed_type_Couch,bed_type_Futon,bed_type_Pull-out Sofa,bed_type_Real Bed,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30,cancellation_policy_super_strict_60,city_Daly City,city_San Francisco,city_South San Francisco,host_response_time_a few days or more,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,market_San Francisco,neighbourhood_cleansed_Bayview,neighbourhood_cleansed_Bernal Heights,neighbourhood_cleansed_Castro/Upper Market,neighbourhood_cleansed_Chinatown,neighbourhood_cleansed_Crocker Amazon,neighbourhood_cleansed_Diamond Heights,neighbourhood_cleansed_Downtown/Civic Center,neighbourhood_cleansed_Excelsior,neighbourhood_cleansed_Financial District,neighbourhood_cleansed_Glen Park,neighbourhood_c

# Split Data into Training and Test Sets

In [88]:
#Prepare data for splitting into train and test sets
X= listings.drop(columns='price').values
y= listings['price'].values

#Check shapes
print('X shape: ', X.shape)
print('y shape: ', y.shape)

X shape:  (65767, 468)
y shape:  (65767,)


In [89]:
#Import models
from sklearn.feature_selection import SelectFromModel, SelectPercentile
from sklearn.model_selection import train_test_split

#Split data
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   random_state=42)

#Check shapes
print('Training data shape: {}'.format(X_train.shape))
print('Test data shape: {}'.format(X_test.shape))

Training data shape: (49325, 468)
Test data shape: (16442, 468)


xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

In [90]:
#Import models
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


# Feature Selection

## Import libraries

In [91]:
#Import ...
#Scaler
from sklearn.preprocessing import StandardScaler

#Feature_selection models
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2,VarianceThreshold

#Remove low variance features
from sklearn.ensemble import ExtraTreesRegressor

#Pipeline
from sklearn.pipeline import Pipeline

#CV
from sklearn.model_selection import cross_validate

## Initial preprocessing pipeline

In [92]:
feature_importance = ExtraTreesRegressor(n_estimators=50, n_jobs=-1,verbose=1,#
                                        random_state=42)

#Create pipeline that scales data, removes low variance feats and extracts feat importances
feature_pipe = Pipeline(steps = [('Normalizer', StandardScaler()),#Step1. Normalize training data
    ('Variance_Selector', VarianceThreshold(threshold=(.8 * (1 - .8)))),#Step2.Remove low variance features
    ('Feature_Importances',feature_importance)]) #Step3.View Feature Importances

#Fit to data
feature_pipe.fit(X_train, y_train)

KeyboardInterrupt: 

## Feature Importances from Initial preprocessor pipeline

In [ ]:
def feature_importance_viz(cols, ft_importances, feat_count, figsize):
    df = pd.DataFrame(list(zip(cols,ft_importances)), columns = ['Feature', 'Relative_Importance'])
    top50 = df.sort_values(by = 'Relative_Importance', ascending = False).head(feat_count)
    ax = top50.plot(kind = 'barh', x='Feature', figsize=figsize,color = 'green') 
    ax.set_title('Top {} Relative Feature Importances'.format(feat_count))
    ax.invert_yaxis()
    
feature_importance_viz(listings.columns, feature_pipe['Feature_Importances'].feature_importances_, 25, (13,6))


## Feature selection and transformation

In [ ]:
#Instantiate SelectFromModel using feature_importance
selector = SelectFromModel(feature_importance, threshold='mean')

#Create pipeline 
preprocessor = Pipeline(steps = [('Normalizer', StandardScaler()),#Step1. Normalize training data
    ('Variance_Selector', VarianceThreshold(threshold=(.8 * (1 - .8)))),#Step2.Remove low variance features
    ('Selector',selector)]) #Step3.Select Features that meet threshold

#Fit to data
preprocessor.fit(X_train, y_train)

In [ ]:
#Transform training and test data
X_train_t = preprocessor.transform(X_train)
X_test_t = preprocessor.transform(X_test)

#View new training/test data shape after transformation
print('New training data shape:', X_train_t.shape)
print('New test data shape:', X_test_t.shape)

# Design base model pipeline

Testing a list of potential regressors that will work well on sparse data. Will optimize best base model.

In [ ]:
#Supervised learning techniques
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

#Create list of regression models to iterate over and test in pipeline
regressors = [RandomForestRegressor(random_state = 42, n_jobs=-1), 
              GradientBoostingRegressor(random_state = 42), 
              BaggingRegressor(random_state = 42, n_jobs=-1),
             MLPRegressor(random_state = 42 ),
             DecisionTreeRegressor(random_state = 42)]

In [ ]:
#Run base models to see which performs best on training data
scoring = ['r2','neg_mean_absolute_error','neg_mean_squared_error']

In [ ]:
for regressor in regressors:
    pipe = Pipeline(steps=[('Model', regressor)], verbose=False) #Fit regressor
    scores  = cross_validate(pipe,X_train_t, y_train,scoring=scoring, cv =5)
    print('=============================================')
    print(regressor)
    print('Mean R2 Score: ', np.mean(scores['test_r2']))
    print('Mean RMSE Score: ', np.mean(np.sqrt(-1 * scores['test_neg_mean_squared_error'])))
    print('Mean MAE Score: ', np.mean(scores['test_neg_mean_absolute_error']))
    print('=============================================')

RandomForestRegressor will be the model of choice. Let's see if we can reduce that average MSE.

# Tune Hyperparameters

## Random Search

In [ ]:
print('Random Forest parameters: ', RandomForestRegressor().get_params)

In [ ]:
#Set values for RandomForestRegressor hyperparameters
max_depth = [20, 30, 40]

max_features= ['auto', 'sqrt', 'log2']

n_estimators = [200, 250, 300]

bootstrap=[True, False]

min_samples_split=[6,8, 10]

min_samples_leaf = [1,3,5,7]

param_grid = {'max_depth' : max_depth,
            'max_features':max_features,
            'n_estimators':n_estimators,
            'bootstrap': bootstrap,
              'min_samples_split':min_samples_split,
            'min_samples_leaf' : min_samples_leaf,
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
RFR_random = RandomizedSearchCV(RandomForestRegressor(random_state = 42, n_jobs=-1), param_distributions= param_grid, n_jobs=-1,
                           scoring = scoring,refit='r2', cv=5,
                                verbose =1)

print('View random_model parameters: ', RFR_random.get_params)

In [ ]:
#Fit model to data
RFR_random.fit(X_train_t, y_train)

In [ ]:
print('Best score from ',RFR_random.best_score_)

print('Best RFR_random params: ', RFR_random.best_params_)

In [ ]:
print('best model parameters: ',RFR_random.best_params_ )
print('Metrics: \nr2: ',np.mean(RFR_random.cv_results_['mean_test_r2']))
print('RMSE: ',np.mean(np.sqrt(-1 * (RFR_random.cv_results_['mean_test_neg_mean_squared_error']))))
print('MAE: ',np.mean(RFR_random.cv_results_['mean_test_neg_mean_absolute_error']))

## Gridsearch

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
#Set values for RandomForestRegressor hyperparameters
max_depth = [25,30,35]

max_features= ['sqrt']

n_estimators = [225,250,275]

bootstrap=[False]

min_samples_split = [5,6]
    
min_samples_leaf = [1,2]

param_grid = {'max_depth' : max_depth,
            'max_features':max_features,
            'n_estimators':n_estimators,
            'bootstrap': bootstrap,
            'min_samples_leaf' : min_samples_leaf,
            'min_samples_split' : min_samples_split
}

In [ ]:
#Create GridSearch
RFR_optimized = GridSearchCV(RandomForestRegressor(random_state = 42, n_jobs=-1), param_grid=param_grid,
                        n_jobs= -1, scoring=scoring,refit='r2',cv=5, verbose=1)

#Fit to data
RFR_optimized.fit(X_train_t, y_train)

In [ ]:
print('best model parameters: ',RFR_optimized.best_params_ )
print('Metrics: \nr2: ',np.mean(RFR_optimized.cv_results_['mean_test_r2']))
print('RMSE: ',np.mean(np.sqrt(-1 * (RFR_optimized.cv_results_['mean_test_neg_mean_squared_error']))))
print('MAE: ',np.mean(RFR_optimized.cv_results_['mean_test_neg_mean_absolute_error']))



## Visualize the results

In [ ]:
#Plot feature importances    
feature_importance_viz(listings.columns, RFR_optimized.best_estimator_.feature_importances_, 45,(15,10))

## Update parameters to optimized values for final model

In [ ]:
#Set values for RandomForestRegressor hyperparameters
max_depth = [35]

max_features= ['sqrt']

n_estimators = [250]

bootstrap=[False]

min_samples_split = [5]
    
min_samples_leaf = [1]

param_grid = {'max_depth' : max_depth,
            'max_features':max_features,
            'n_estimators':n_estimators,
            'bootstrap': bootstrap,
            'min_samples_leaf' : min_samples_leaf,
            'min_samples_split' : min_samples_split
}

#Update model with optimized hyperparameters
RFR_optimized = GridSearchCV(RandomForestRegressor(random_state = 42, n_jobs=-1), param_grid=param_grid,
                        n_jobs= -1, scoring=scoring,refit='r2', verbose=1, cv=10)

#Check
print('Finalized model: \n', RFR_optimized)

# Write Files and Model

In [ ]:
from sklearn.externals import joblib

#Set path to write test data
path =  r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\04_Models'

# save the model to disk
filename = '/2020_0128_Price_Predictor_Final.sav'
joblib.dump(RFR_optimized, path + filename)

In [ ]:
#Set path to write test data
path =  r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Air BnB - SF\Data\04_Models'

#Write Test data to csv
pd.DataFrame(X_test_t).to_csv(path + '/2020_0128_Test_Data.csv', index=None)
pd.DataFrame(y_test).to_csv(path + '/2020_0128_Test_Labels.csv', index=None)